In [1]:
import ee
import os
import pandas as pd
from datetime import datetime as dt
from pathlib import Path
from abd import abd

In [2]:
__version__ = ""
exec(Path(f"{os.getcwd()}/src/abd/version.py").read_text(encoding="utf-8"))

In [3]:
# Initialize Google Earth Engine
# ee.Authenticate() (might be needed in some situations)
ee.Initialize()

In [4]:
# Data path
folderRoot = os.getcwd() + "/results"
if not os.path.exists(folderRoot):
    os.mkdir(folderRoot)

# Images path
folderCache = os.getcwd() + "/cache"
if not os.path.exists(folderCache):
    os.mkdir(folderCache)
    
# Selected image date to dectection
selectedDate = "2019-06-03"

# folder to save results from algorithm at
folder = (
    folderRoot
    + "/"
    + dt.now().strftime("%Y%m%d_%H%M%S")
    + f"[v={str(__version__)}-sandbox,d={selectedDate},t=180,m=ocsvm,s=modis]")
if not os.path.exists(folder):
    os.mkdir(folder)

In [5]:
# fix permissions
!chmod 777 $folderRoot
!chmod 777 $folder
!chmod 777 $folderCache

zsh:1: no matches found: /Users/phmananias/Documents/unesp/abd/results/20221211_130351[v=v0.29.0-sandbox,d=2019-06-03,t=180,m=ocsvm,s=modis]


In [6]:
# create algorithm object
algorithm = abd.Abd(lat_lon="-83.50124371805877,41.88435023280987,-83.07548096199702,41.65275061592091",
                    date=selectedDate,
                    days_threshold=180,
                    model="ocsvm",
                    sensor="modis",
                    cache_path=folderCache)

2022-12-11 13:03:51.541 | DEBUG    | abd.abd:__init__:205 - Selected sensor: Modis MOD09GA.006 [GEE Dataset = MODIS/006/MOD09GA]
2022-12-11 13:03:55.228 | DEBUG    | abd.abd:__init__:377 - Check if requested date is good for processing [2019-06-03]...
2022-12-11 13:03:55.230 | DEBUG    | abd.abd:__init__:386 - Extracting geometry (0) 1 of 1...
2022-12-11 13:03:55.712 | DEBUG    | abd.abd:__init__:438 - Pixel and cloud score for geometry #1: 1.0 and 0.73926!
2022-12-11 13:03:55.713 | DEBUG    | abd.abd:__init__:454 - Ok, image is good for processing: 1
2022-12-11 13:03:56.657 | DEBUG    | abd.abd:__init__:459 - Statistics: scale=500 meters, pixels=4845, date_start='2018-12-05', date_end='2019-11-30', interval_images='360', interval_unique_images='360', water_mask_images='16', days_threshold='180', morph_op='None', morph_op_iters='1', convolve='False', convolve_radius='1', scaler='robust', model='ocsvm', outliers_zscore='3.0', attributes='['cloud', 'ndvi', 'fai']', attribute_doy='False'


In [ ]:
# creating timeseries based on days_thresholds
algorithm.process_timeseries_data()

2022-12-11 13:03:56.675 | DEBUG    | abd.abd:process_timeseries_data:1043 - Starting time series processing ...
2022-12-11 13:03:56.690 | DEBUG    | abd.abd:process_timeseries_data:1055 - Trying to extract it from the cache...
2022-12-11 13:03:56.694 | ERROR    | abd.abd:process_timeseries_data:1071 - Error trying to get it from cache: either doesn't exist or is corrupted! Creating it again...
2022-12-11 13:04:04.588 | DEBUG    | abd.abd:extract_image_pixels:764 - Processing date [2018-12-05]...
2022-12-11 13:04:04.595 | DEBUG    | abd.abd:extract_image_pixels:806 - Extracting geometry (0) 1 of 1...
2022-12-11 13:04:04.609 | DEBUG    | abd.abd:extract_image_pixels:764 - Processing date [2018-12-20]...
2022-12-11 13:04:04.611 | DEBUG    | abd.abd:extract_image_pixels:764 - Processing date [2018-12-17]...
2022-12-11 13:04:04.623 | DEBUG    | abd.abd:extract_image_pixels:764 - Processing date [2018-12-08]...
2022-12-11 13:04:04.624 | DEBUG    | abd.abd:extract_image_pixels:806 - Extractin

In [ ]:
# creating train and grid datasets
algorithm.process_training_data(df=algorithm.df_timeseries)

In [ ]:
# start training process
algorithm.train()

In [ ]:
# apply detection algorithm
algorithm.detect()

In [ ]:
# save geojson
algorithm.save_geojsons(folder=folder+"/geojson")

In [ ]:
# save results
algorithm.save_dataset(df=algorithm.df_results, path=folder+'/results.csv')

In [ ]:
# save detection plot
algorithm.save_detection_plot(path=folder+'/detection.png')